# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |Maria Fernanda Muñoz Sevilla   |
| **Fecha**      15/09/25|   |
| **Expediente** 751190|   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Porque nos ayuda a separar nuestros datos en dos, en train (entrenamiento) y test (prueba), para lograr un mejor análisis de nuestros datos

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

No, porque la población que tiene una gran cantidad de datos, lo que no nos permitiría generalizar los datos porque no representa una muestra sino el total de todo. Es por esto que, al utilizar una muestra nos permite manejar mejor los datos para su validación

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [56]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [6]:
df=pd.read_excel("C:/Users/munoz/Downloads/lab_apre_est/Motor Trend Car Road Tests.xlsx")
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [7]:
df2=df.drop(columns=["model"])
df2.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [8]:
mses = []

for i in range(32):

    dftrain = df2.drop(index=i)
    dftest = df2.iloc[[i]]
    
    X_train = dftrain.drop(columns=["mpg"])
    y_train = dftrain["mpg"].values
    X_test = dftest.drop(columns=["mpg"])
    y_test = dftest["mpg"].values

    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_train_sm = sm.add_constant(X_train_scaled)
    
    modelo_mpg = sm.OLS(y_train, X_train_sm).fit()
 
    X_test_scaled = scaler.transform(X_test)
    X_test_sm = sm.add_constant(X_test_scaled, has_constant='add')
    y_pred = modelo_mpg.predict(X_test_sm)
    
    mse = mean_squared_error(y_test, y_pred)
    mses.append(mse)

mses
print("El MSE promedio es: ", np.mean(mses))
print("La desviación estándar del MSE es: ", np.std(mses))


El MSE promedio es:  12.181558006901938
La desviación estándar del MSE es:  17.067399871888526


Interpreta.

El modelo tiene un desempeño razonable en promedio, pero sus predicciones son inestables

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [142]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [162]:
#Convierto en dataframe nuestros datos, tomo la coulmna de X que es Features 
df = pd.DataFrame(X, columns=housing.feature_names)
#Agrego en mi dataset los datos de target
df['MedHouseVal'] = y
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [164]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [148]:
#Lo descargo para Pao
df.to_excel("housing.xlsx", index=False)

In [166]:
df_shuffled = df.sample(frac=1, random_state=137)
df_shuffled.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
17500,5.1615,26.0,6.687037,1.042593,2089.0,3.868519,34.43,-119.79,2.762
18701,3.2639,22.0,5.504673,0.906542,338.0,3.158879,40.57,-122.35,0.875
1738,4.4135,32.0,5.403846,0.956044,1144.0,3.142857,37.98,-122.33,1.501
7984,3.7527,38.0,4.410596,1.036424,1627.0,2.693709,33.87,-118.18,1.619
14857,1.8672,43.0,3.988095,0.912698,548.0,2.174603,32.64,-117.08,1.458


In [172]:
mses = []
n_split = 10
n = len(y)
fold_size = n // n_split

for i in range(n_split):
    X_train = np.concatenate([X[:i*fold_size], X[(i+1)*fold_size:]])
    y_train = np.concatenate([y[:i*fold_size], y[(i+1)*fold_size:]])
    X_test = X[i*fold_size:(i+1)*fold_size]
    y_test = y[i*fold_size:(i+1)*fold_size]

    #Estandarizamos
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entrenamos el modelo
    model = sm.OLS(y_train, sm.add_constant(X_train)).fit()

    # calculamos el mse
    y_pred = model.predict(sm.add_constant(X_test))
    mse = mean_squared_error(y_test, y_pred)
    mses.append(mse)


print("La desviación estándar del MSE es: ", np.std(mses))
print("El MSE promedio es: ", np.mean(mses))

La desviación estándar del MSE es:  0.19288582953892686
El MSE promedio es:  0.5509524296956589


Interpreta.

Ya podemos generalizar

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3